In [35]:
import torch

# 예시는 2x2 이미지, 배치 사이즈 2
pred = torch.tensor([[[1,2],[4,1]],[[3,1],[4,1]]])
gt = torch.tensor([[[2,3],[3,1]],[[2,1],[4,1]]])


def loss_function(pred,y,cutMix=False,feature_alignment=False,f1=None,f2=None):
    """
    :param pred: Prediction per pixel. size : BxHxW
    :param y: Ground truth. size : BxHxW
    :param perturbation: For unlabeled (pseudo) dataset
    :param feature_alignment: For Idea comparison
    :param f1,f2: latent vector from encoder that contains high representation per patch << 이거 논문에서는 그냥 loss 함수에 동일한 HW 썼지만은, 정황상 H W는 패치로 주는게 맞음 -> 이거 size [ B, num_P, embedding ]
    :return:

    """
    B,H,W = pred.shape

    #pred = torch.reshape(pred,(B,H*W))
    #y = torch.reshape(y,(B,H*W))

    pred = pred.view(B, H * W)
    y = y.view(B, H * W)
    loss_l= 0
    loss_u= 0
    loss_feat = 0

    def d_hat(d):
        # 각 배치별 계산을 위해 dim 설정
        # 여기서는 B x (HxW) 로 2차원이라고 가정
        median, _ = torch.median(d, dim=-1) # 이러면 나오는 결과 : Bx1
        # print(median)
        t = median.unsqueeze(-1)
        t = t.expand(-1,d.shape[-1])

        #t = torch.matmul(median.unsqueeze(0),t)
        s = torch.sum(torch.abs(d-t),dim=-1)/(d.shape[-1])
        # s : Bx1 사이즈

        s = s.unsqueeze(-1).expand(-1,d.shape[-1])

        #print("d : ",d)
        #print("t : ",t)
        #print("s : ",s)
        #print("분자 : ",torch.sum(d-t,dim=-1))

        return (d-t)/s

    def rho(pred,y):
        return torch.abs(d_hat(pred) - d_hat(y))

## 이미 배치 구성 단계에서 저걸 포함시켰어야한,.??
    def cutmix(u_cutmix,u_normal_a,u_normal_b,mask):
        """
        :param u_cutmix: cutmix된 이미지 -> student에게 먹일 데이터
        :param u_normal: cutmix 안된 일반 이미지 -> teacher 에게 먹일 데이터
        :param mask: u_whole과 u_part가 공유하는 부분만 1이고, 나머지는 0인 마스킹
        :return: loss_u

        < instruction >
        """

        mask_flat = mask.view(B, H * W).float()
        inv_mask = 1.0 - mask_flat

        # 영역별 loss: 여기서는 rho를 이용하여 픽셀별 loss
        ##### 이렇게 하면ㅇ ㅏ안돼 1!!!!! 왜ㅔ야냐하면 이미 우리 데이터 배치단위로 넣었다고 가정하잖아

        loss_u_a = torch.mean(rho(pred * mask_flat, u_normal_a.view(B, H * W) * mask_flat), dim=-1)
        loss_u_b = torch.mean(rho(pred * inv_mask, u_normal_b.view(B, H * W) * inv_mask), dim=-1)

        return torch.mean(loss_u_a + loss_u_b)


    if cutMix:

        loss_u = cutmix(u_cutmix, u_normal_a, u_normal_b, mask)

        if feature_alignment and f1 and f2 :
            """
            f1 f2 간의 cos similarity 도입
            여기서 f1는 student model의 encoder에서 나온 결과, f2는 frozen dino에서 나온 결과

            고로 f1에는 perturbation이 적용되었음 -> 단 gaussian blur과 jittering같은 것만 포함

            size [ B, num_P, embedding ]
            """
            # f1 , f2 : B x P x embedding 사이즈
            _ , num_patch, dim_embedding = f1.shape

            dot_product = torch.sum(torch.mul(f1,f2),dim=-1) # Bx1 사이즈
            size_f1 = torch.sqrt(torch.sum(torch.mul(f1,f1),dim=-1))
            size_f2 = torch.sqrt(torch.sum(torch.mul(f2,f2),dim=-1))
            denominator = torch.mul(size_f1,size_f2) # Bx1 사이즈

            loss_feat = torch.sum(1-(dot_product/denominator)/num_patch)

    else :
        loss_l = torch.sum(rho(pred,y),dim=-1) / pred.shape[-1]

    return loss_l + loss_u + loss_feat

print(loss_function(pred,gt))

tensor([0.8333, 0.3000])
